In [1]:
import os
import pyspark

conf = pyspark.SparkConf()
conf = conf.setAppName("<my-app-name>")
conf.set('spark.ui.proxyBase', '/user/' + os.environ['JUPYTERHUB_USER'] + '/proxy/4040') ## to setup SPARK UI
conf = conf.set('spark.jars', os.environ['GRAPHFRAMES_PATH']) ## graphframes in spark configuration
sc = pyspark.SparkContext(conf=conf)
sc

24/10/23 01:00:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


<SparkContext master=local[*] appName=<my-app-name>>

In [2]:
#QUESTION 1 :  25 Points, Data : "shared/data/Bakert.csv" 

from pyspark.sql import SparkSession
sc.setLogLevel("ERROR")
spark = SparkSession(sc)
df = spark.read.option("header", "true").csv("shared/data/Bakery.csv")
df.show()

+----------+--------+-----------+-------------+
|      Date|    Time|Transaction|         Item|
+----------+--------+-----------+-------------+
|2016-10-30|09:58:11|          1|        Bread|
|2016-10-30|10:05:34|          2| Scandinavian|
|2016-10-30|10:05:34|          2| Scandinavian|
|2016-10-30|10:07:57|          3|Hot chocolate|
|2016-10-30|10:07:57|          3|          Jam|
|2016-10-30|10:07:57|          3|      Cookies|
|2016-10-30|10:08:41|          4|       Muffin|
|2016-10-30|10:13:03|          5|       Coffee|
|2016-10-30|10:13:03|          5|       Pastry|
|2016-10-30|10:13:03|          5|        Bread|
|2016-10-30|10:16:55|          6|    Medialuna|
|2016-10-30|10:16:55|          6|       Pastry|
|2016-10-30|10:16:55|          6|       Muffin|
|2016-10-30|10:19:12|          7|    Medialuna|
|2016-10-30|10:19:12|          7|       Pastry|
|2016-10-30|10:19:12|          7|       Coffee|
|2016-10-30|10:19:12|          7|          Tea|
|2016-10-30|10:20:51|          8|       

In [3]:
from pyspark.sql.functions import col, date_format, hour
df = df.withColumn('Date', col('Date').cast('date'))
df = df.withColumn('weekday', date_format(col('Date'), 'EEEE'))
df = df.withColumn('Hour', hour(col('Time')))
df.show()
df_monday = df.filter(col('weekday') == 'Monday')
df_monday.show()


+----------+--------+-----------+-------------+-------+----+
|      Date|    Time|Transaction|         Item|weekday|Hour|
+----------+--------+-----------+-------------+-------+----+
|2016-10-30|09:58:11|          1|        Bread| Sunday|   9|
|2016-10-30|10:05:34|          2| Scandinavian| Sunday|  10|
|2016-10-30|10:05:34|          2| Scandinavian| Sunday|  10|
|2016-10-30|10:07:57|          3|Hot chocolate| Sunday|  10|
|2016-10-30|10:07:57|          3|          Jam| Sunday|  10|
|2016-10-30|10:07:57|          3|      Cookies| Sunday|  10|
|2016-10-30|10:08:41|          4|       Muffin| Sunday|  10|
|2016-10-30|10:13:03|          5|       Coffee| Sunday|  10|
|2016-10-30|10:13:03|          5|       Pastry| Sunday|  10|
|2016-10-30|10:13:03|          5|        Bread| Sunday|  10|
|2016-10-30|10:16:55|          6|    Medialuna| Sunday|  10|
|2016-10-30|10:16:55|          6|       Pastry| Sunday|  10|
|2016-10-30|10:16:55|          6|       Muffin| Sunday|  10|
|2016-10-30|10:19:12|   

In [4]:
from pyspark.sql.functions import count
df_monday_filtered = df_monday.filter((col('Hour') >= 7) & (col('Hour') <= 11))
df_grouped = df_monday_filtered.groupBy("Date", "Hour", "Item").agg(count("Item").alias("total_quantity"))
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
windowSpec = Window.partitionBy("Date", "Hour").orderBy(col("total_quantity").desc())
df_with_row_num = df_grouped.withColumn("rank", row_number().over(windowSpec))
df_top_items = df_with_row_num.filter(col("rank") == 1).drop("rank")
df_top_items.show(truncate=False)

+----------+----+------+--------------+
|Date      |Hour|Item  |total_quantity|
+----------+----+------+--------------+
|2016-10-31|8   |Coffee|2             |
|2016-10-31|9   |Coffee|11            |
|2016-10-31|10  |Coffee|10            |
|2016-10-31|11  |Coffee|13            |
|2016-11-07|8   |Pastry|1             |
|2016-11-07|9   |Bread |3             |
|2016-11-07|10  |Coffee|7             |
|2016-11-07|11  |Coffee|10            |
|2016-11-14|7   |Coffee|1             |
|2016-11-14|8   |Coffee|2             |
|2016-11-14|9   |Coffee|5             |
|2016-11-14|10  |Coffee|5             |
|2016-11-14|11  |Bread |5             |
|2016-11-21|7   |Coffee|1             |
|2016-11-21|8   |Coffee|2             |
|2016-11-21|9   |Coffee|8             |
|2016-11-21|10  |Coffee|4             |
|2016-11-21|11  |Coffee|4             |
|2016-11-28|7   |Coffee|1             |
|2016-11-28|8   |Coffee|1             |
+----------+----+------+--------------+
only showing top 20 rows



In [5]:
# QUESTION 2 : 25 Points, Data = "shared/data/Bakery.csv"

from pyspark.sql.functions import col, when, row_number, count, collect_list, concat_ws
from pyspark.sql.window import Window

df = df.withColumn("DayType", when(col("weekday").isin("Saturday", "Sunday"), "Weekend").otherwise("Weekday"))

df = df.withColumn("Daypart", when((col("hour") >= 6) & (col("hour") < 11), "Breakfast")
                                .when((col("hour") >= 11) & (col("hour") < 16), "Lunch")
                                .otherwise("Dinner"))
grouped_df = df.groupBy("DayType", "Daypart", "Item").agg(count("Item").alias("total_qty"))
window_spec = Window.partitionBy("DayType", "Daypart").orderBy(col("total_qty").desc())

ranked_df = grouped_df.withColumn("rank", row_number().over(window_spec)) \
                      .filter(col("rank") <= 2) \
                      .drop("rank")
result = ranked_df.groupBy("DayType", "Daypart").agg(concat_ws(", ", collect_list("Item")).alias("Top_2_Items"))
result.show(truncate=False)


+-------+---------+-------------+
|DayType|Daypart  |Top_2_Items  |
+-------+---------+-------------+
|Weekday|Breakfast|Coffee, Bread|
|Weekday|Dinner   |Coffee, Bread|
|Weekday|Lunch    |Coffee, Bread|
|Weekend|Breakfast|Coffee, Bread|
|Weekend|Dinner   |Coffee, Bread|
|Weekend|Lunch    |Coffee, Bread|
+-------+---------+-------------+



In [6]:
# QUESTION 3  20 Points , Data = "shared/data/Restaurants_in_Durham_County_NC.json"

df = spark.read.json("shared/data/Restaurants_in_Durham_County_NC.json")
grouped_df = df.groupBy("fields.rpt_area_desc").count()
grouped_df.show(truncate=False)

+---------------------+-----+
|rpt_area_desc        |count|
+---------------------+-----+
|Bed&Breakfast Home   |4    |
|Summer Camps         |4    |
|Institutions         |30   |
|Local Confinement    |2    |
|Mobile Food          |147  |
|School Buildings     |89   |
|Summer Food          |242  |
|Swimming Pools       |420  |
|Day Care             |173  |
|Tattoo Establishments|36   |
|Residential Care     |154  |
|Bed&Breakfast Inn    |2    |
|Adult Day Care       |5    |
|Lodging              |62   |
|Food Service         |1093 |
+---------------------+-----+



In [7]:
#QUESTION 4 20 Points, Data = "shared/data/populationbycountry19802010millions.csv"

from pyspark.sql import SparkSession
from pyspark.sql import functions as F

file_path = "shared/data/populationbycountry19802010millions.csv"
data = spark.read.csv(file_path, header=True, inferSchema=True)
cleaned_data = data.toDF(*(c.strip() for c in data.columns))
population_data = cleaned_data.select("_c0", "1990", "2000").na.drop()

population_data = population_data.withColumnRenamed("_c0", "Country Name")
population_change = population_data.withColumn(
    "Percentage Change",
    ((F.col("2000") - F.col("1990")) / F.col("1990")) * 100
).filter(F.col("Percentage Change").isNotNull())

max_increase = population_change.orderBy(F.col("Percentage Change").desc()).first()
max_decrease = population_change.orderBy(F.col("Percentage Change").asc()).first()
print(f"{max_increase['Country Name']}, {max_increase['Percentage Change']:.2f}%")
print(f"{max_decrease['Country Name']}, {max_decrease['Percentage Change']:.2f}%")
print(max_increase)
print(max_decrease)

United Arab Emirates, 76.28%
Montserrat, -63.19%
Row(Country Name='United Arab Emirates', 1990='1.82625', 2000='3.2193', Percentage Change=76.27926078028749)
Row(Country Name='Montserrat', 1990='0.01073', 2000='0.00395', Percentage Change=-63.18732525629077)


In [8]:
# QUESTION 5 20 POINTS, DATA : HW1 TEXT - WQRD COUNT

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.feature import RegexTokenizer
from pyspark.ml import Pipeline


spark = SparkSession.builder \
    .appName("Word Count") \
    .getOrCreate()

file_paths = ["20-01.txt", "20-02.txt", "20-03.txt", "20-04.txt", "20-05.txt"]
global_word_count = {}

for file_path in file_paths:
    data = spark.read.text(file_path)
    normalized_data = data.select(F.lower(F.regexp_replace("value", "[^0-9a-z]", " ")).alias("normalized_text"))
    tokenizer = RegexTokenizer(inputCol="normalized_text", outputCol="words", pattern="\\s+")
    pipeline = Pipeline(stages=[tokenizer])
    model = pipeline.fit(normalized_data)
    tokenized_data = model.transform(normalized_data)
    exploded_data = tokenized_data.select(F.explode(F.col("words")).alias("word"))
    word_count = exploded_data.groupBy("word").count()
    for row in word_count.collect():
        word = row['word']
        count = row['count']
        if word in global_word_count:
            global_word_count[word] += count
        else:
            global_word_count[word] = count

global_word_count_df = spark.createDataFrame(global_word_count.items(), ["word", "count"])
global_word_count_df = global_word_count_df.orderBy(F.desc("count"))
global_word_count_df.show(truncate=False)
spark.stop()



+----+------+
|word|count |
+----+------+
|the |142965|
|to  |87873 |
|p   |78583 |
|of  |75074 |
|and |70933 |
|in  |52844 |
|a   |50187 |
|for |28369 |
|he  |27781 |
|is  |27646 |
|that|27443 |
|s   |25354 |
|on  |23636 |
|are |19529 |
|with|18699 |
|be  |17764 |
|as  |16110 |
|have|16083 |
|at  |15209 |
|said|14893 |
+----+------+
only showing top 20 rows



In [9]:
# QUESTION 6 : 20 POINTS, DATA = HW 1 - TOP 10 MOST COMMON BIGRAMS

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.ml.feature import NGram

spark = SparkSession.builder \
    .appName("Bigram Count") \
    .getOrCreate()

files = ["20-01.txt", "20-02.txt", "20-03.txt", "20-04.txt", "20-05.txt"]
global_bigram_counts = {}

for file_path in files:
    text_data = spark.read.text(file_path)
    cleaned_data = text_data.select(
        F.lower(F.regexp_replace('value', '[^0-9A-Za-z]', ' ')).alias('cleaned_text')
    )

    tokenized_data = cleaned_data.select(
        F.split(F.col('cleaned_text'), ' ').alias('words')
    )

    bigram = NGram(n=2, inputCol='words', outputCol='bigrams')
    bigram_data = bigram.transform(tokenized_data)
    exploded_bigrams = bigram_data.select(F.explode('bigrams').alias('bigram'))
    filtered_bigrams = exploded_bigrams.filter(F.col('bigram').rlike('^[a-z0-9]+ [a-z0-9]+$'))

    bigram_counts = filtered_bigrams.groupBy('bigram').count()
    
    for row in bigram_counts.collect():
        bigram = row['bigram']
        count = row['count']
        if bigram in global_bigram_counts:
            global_bigram_counts[bigram] += count
        else:
            global_bigram_counts[bigram] = count

global_top_bigrams = sorted(global_bigram_counts.items(), key=lambda x: x[1], reverse=True)[:10]
formatted_output = [f"{bigram} : {count}" for bigram, count in global_top_bigrams]

for result in formatted_output:
    print(result)

spark.stop()


of the : 17436
in the : 12777
covid 19 : 8726
to the : 8331
for the : 5573
n t : 5393
on the : 4990
to be : 4571
will be : 4175
and the : 4026


In [10]:
# QUESTION 7 - BONUS QUESTION - 35 POINTS - HAVERSINE 

from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from haversine import haversine, Unit
from pyspark.sql.types import DoubleType

spark = SparkSession.builder \
    .appName("Find Restaurants and Foreclosures") \
    .getOrCreate()

foreclosure_file = "shared/data/durham-nc-foreclosure-2006-2016.json"
restaurants_file = "shared/data/Restaurants_in_Durham_County_NC.json"
foreclosure_df = spark.read.json(foreclosure_file)
restaurants_df = spark.read.json(restaurants_file)

active_restaurants_df = restaurants_df.filter(
    (restaurants_df.fields.status == "ACTIVE") & 
    (restaurants_df.fields.rpt_area_desc == "Food Service")
)

reference_point = (35.994914, -78.897133)

def calculate_distance(lat1, lon1, lat2, lon2):
    if lat1 is None or lon1 is None or lat2 is None or lon2 is None:
        return None
    return haversine((lat1, lon1), (lat2, lon2), unit=Unit.MILES)

calculate_distance_udf = F.udf(calculate_distance, DoubleType())

active_restaurants_with_distance_df = active_restaurants_df.withColumn(
    "distance", 
    calculate_distance_udf(
        F.lit(reference_point[0]), 
        F.lit(reference_point[1]), 
        active_restaurants_df.fields.geolocation[0], 
        active_restaurants_df.fields.geolocation[1]
    )
).filter(F.col("distance").isNotNull())

closest_restaurant_df = active_restaurants_with_distance_df.orderBy("distance").limit(1)

for row in closest_restaurant_df.collect():
    restaurant_name = row['fields'].premise_name
    geolocation = row['fields'].geolocation
    distance = row['distance']
    print(f"Part 1 Answer: \n\n")
    print(f"Restaurant Name: {restaurant_name}")
    print(f"Geolocation: Latitude {geolocation[0]}, Longitude {geolocation[1]}")
    print(f"Distance from Reference Point: {distance:.2f} miles")
    print("\n" + "-" * 80 + "\n")

closest_restaurant_coords = closest_restaurant_df.select(
    active_restaurants_df.fields.geolocation[0].alias("latitude"),
    active_restaurants_df.fields.geolocation[1].alias("longitude")
).first()

if closest_restaurant_coords:
    latitude, longitude = closest_restaurant_coords
    print(f"The Latitude is {latitude} and the Longitude is {longitude}")

    foreclosures_with_distance_df = foreclosure_df.withColumn(
        'distance_to_restaurant',
        calculate_distance_udf(
            F.lit(latitude), 
            F.lit(longitude), 
            foreclosure_df.fields.geocode[0], 
            foreclosure_df.fields.geocode[1]
        )
    ).filter(F.col('distance_to_restaurant').isNotNull())

    foreclosures_within_radius_df = foreclosures_with_distance_df.filter(
        F.col('distance_to_restaurant') <= 1.0
    )

    foreclosure_count = foreclosures_within_radius_df.count()
    print(f"Number of foreclosures within 1 mile of the closest restaurant: {foreclosure_count}\n")
    print("Distances to Foreclosures (VERIFICATION PRINTING IN ASCENDING ORDER):")
    
    foreclosures_with_distance_df.select(
        'fields.geocode', 
        'distance_to_restaurant'
    ).orderBy('distance_to_restaurant', ascending=True).show(truncate=False)

spark.stop()


Part 1 Answer: 


Restaurant Name: OLD HAVANA SANDWICH SHOP
Geolocation: Latitude 35.9932826, Longitude -78.8981331
Distance from Reference Point: 0.13 miles

--------------------------------------------------------------------------------



The Latitude is 35.9932826 and the Longitude is -78.8981331
Number of foreclosures within 1 mile of the closest restaurant: 320

Distances to Foreclosures (VERIFICATION PRINTING IN ASCENDING ORDER):


+-------------------------+----------------------+
|geocode                  |distance_to_restaurant|
+-------------------------+----------------------+
|[35.995413, -78.8950321] |0.2274148959935075    |
|[35.995797, -78.895396]  |0.2315019166859024    |
|[35.9965639, -78.9004693]|0.2616405140108075    |
|[35.9961544, -78.8929116]|0.3529462418852746    |
|[35.9961544, -78.8929116]|0.3529462418852746    |
|[35.995954, -78.892736]  |0.3536878179799853    |
|[35.995954, -78.892736]  |0.3536878179799853    |
|[35.9968123, -78.8935006]|0.3557226602418183    |
|[35.9968123, -78.8935006]|0.3557226602418183    |
|[35.9988695, -78.8990179]|0.389173850545146     |
|[35.9988695, -78.8990179]|0.389173850545146     |
|[35.9988695, -78.8990179]|0.389173850545146     |
|[35.995823, -78.891795]  |0.3954043208699746    |
|[35.995823, -78.891795]  |0.3954043208699746    |
|[35.995823, -78.891795]  |0.3954043208699746    |
|[35.995582, -78.8913248] |0.41242418898431704   |
|[35.995582, -78.8913248] |0.41